# GPT2でファインチューニングしたモデルから文章生成する。

公開されているRinnaの学習モデルをファインチューニングしたモデルから文章の生成を行う。

環境

 - Clab PRO
 - Google Drive

ディレクトリ

./output_sangoku : 学習したモデルを保存

In [33]:
# Google Driveに接続
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/work


In [34]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Runtime 再起動

In [35]:
# 作業フォルダへ
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


In [36]:
# Huggingface Datasetsのインストール
!pip install datasets==1.2.1
# Sentencepieceのインストール
!pip install sentencepiece==0.1.91

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


「./transformers/examples/language-modeling/run_clm.py」の編集。
「rinna」の日本語GPT-2モデルは「AutoTokenizer」ではなく「T5Tokenizer」を使う必要があります。

In [37]:
from transformers import T5Tokenizer, AutoModelForCausalLM
import re

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
model = AutoModelForCausalLM.from_pretrained("output_sangoku/")

# テスト生成
input = tokenizer.encode(
    """劉備は孔明に問うた。""", return_tensors="pt")
output = model.generate(input,
                        do_sample=True,
                        top_p=0.95,
                        top_k=0,
                        max_length=400,
                        skip_special_tokens=True,
                        pad_token_id=tokenizer.pad_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bad_word_ids=[[tokenizer.unk_token_id]],
                        num_return_sequences=20
                        )

# 結果表示
for list_output in output:
  print(re.sub(r'<s>|<unk>|\[PAD\]|\[SEP\]| ', '',
               tokenizer.decode(list_output).replace('</s>', '\n')))


劉備は孔明に問うた。
「いやしくも申したり。其許は疾くより何故、この地に来り、賢慮をめぐらして、ここの守りをいいつけたか」

劉備は孔明に問うた。
「さきに張魯様にも、やはり同じようなご苦衷を語られていましたな」

劉備は孔明に問うた。
「関羽の戦法は、新手ですな。何が違うのか」

劉備は孔明に問うた。
「関羽の戦法は、まるで児童戦のようです。なぜならば、私にはその戦法を工夫する才識がありませんから」

劉備は孔明に問うた。
「隴西の諸郡から、長安へむかって、勝ちに乗ってくる精兵は、何と約数のいわれを持っているか」

劉備は孔明に問うた。
「では、馬超の身内は、やはりあの身内でないことになってきたな」

劉備は孔明に問うた。
「丞相、昨夜から今朝にかけて、夢に怪異な夢を幾度か見たのですが」

劉備は孔明に問うた。
「では、甲まで出していたんですか?」

劉備は孔明に問うた。
「では、汝には、後堂にいる良人に向いて、そっと剣を渡してくれ。その剣で汝にも、禍いを転嫁してやろう」

劉備は孔明に問うた。
「では、三度勅使を待つつもりか」

劉備は孔明に問うた。
「では、汝の弟は、どの辺にいるのか」

劉備は孔明に問うた。
「かかる大雪。この路に立って、軍旅をするあなたがたには、何の苦心もなく、無事に帰るつもりですか」

劉備は孔明に問うた。
「荊州の形勢はどうだった?」

劉備は孔明に問うた。
「関羽の武勇は、古の孫子呉子にも劣らないものだ。いま、孫呉のを破ったものは、貴公たち二人で、赤壁の一戦に、勝ったものは、貴公たち二人で、その勲功は、十万に近いであろう」

劉備は孔明に問うた。
「勝敗は兵家の常だ。どうしてそうなるのか」

劉備は孔明に問うた。
「丞相が即位なされたら、いつご狩猟にお出まし遊ばしますか」

劉備は孔明に問うた。
「では、汝の如きは、そもそも、なんの能があるのか」

劉備は孔明に問うた。
「丞相がよくご存じなのは、かの虎が死んだ時、いかにして関羽を得たかということです。故人の遺託をうけて、これを丞相に献じ、丞相は、関羽を亡ぼしてから、ふたたび荊州に亡父の墳墓を営まれることになろうとは、人臣のの道でも、あまりよろしくありません。では、丞相は、いかなる計をもって、関羽を亡ぼしたか」

劉備は孔明に問うた。
「では、大都督には、すでにご着任とみえるな。何の急

In [38]:
# 生成部

def generate_txt(prompt_txt) -> str:
  '''テキスト生成'''
  input = tokenizer.encode(
    prompt_txt, return_tensors="pt")
  output = model.generate(input,
                          do_sample=True,
                          top_p=0.95,
                          top_k=0,
                          max_length=1024,
                          skip_special_tokens=True,
                          pad_token_id=tokenizer.pad_token_id,
                          bos_token_id=tokenizer.bos_token_id,
                          eos_token_id=tokenizer.eos_token_id,
                          bad_word_ids=[[tokenizer.unk_token_id]],
                          num_return_sequences=1
                          )
  return tokenizer.decode(output[0],skip_special_tokens=True)

prompt_txt = "劉備は孔明に問うた。"
print(generate_txt(prompt_txt))

劉備は孔明に問うた。 「丞相の兵馬は、みな東方の山岳におわす。丞相の発向したその方向へ、兵を進めては遅くないでしょう」


In [41]:
# 作文
prompt_txt = "劉備は孔明に問うた。"

#
lines = [prompt_txt]
i = 0

while i < 100:
  i += 1
  line = generate_txt(prompt_txt)
  print(line[len(prompt_txt):])
  lines.append(line[len(prompt_txt):])
  prompt_lines = len(lines)
  prompt_lines -= 10
  if prompt_lines < 0:
    prompt_lines = 0
  prompt_txt = "\n".join(lines[prompt_lines:])
  # print("prompt",prompt_txt)


 「ますら、貴公はこの一戦のさなか、いかなる妙策をめぐらしたか」
「さればです。思い出せば、二十年ほど前、知人の秦川から、一羽の鹿を贈ってきました。たいへん稀有な鹿で、羽つきで舌ばしりも強く、手に立てても非常に重く、五百斤もおうろうかと思えるものでしたが、朽ちても朽ちても草木を食いますので、値につけては皆手もとになく、とうとう曹操にあげてしまったことがあるのです」
「ふむ。そういわれてみると、なお考えれば考えるほど、これは奇妙だが、大いなる計策でもあろうか」
「さればです。ふと考えたのですが、無名の英傑をあげて、その人物を、ひそかに貴公の麾下へ加え、時々、上庸の城へ呼んで、狩猟をさせたら、どんなことになるかと思いますが」
「なるほど、これは一計だ。さっそく明日、それがしを城へ呼んで、親しく其方と狩猟を共にしては如何」
孔明は、すぐ仮の官渡城へ移った。そして毎日、外敵の糧食毒酒など毒に浸した檻車を、猛獣の群れが通りそうな岸へ上せて、何十台となく車を護岸した。
孔明の「善策」を聞いて、軍はたちまち意気を昂げた。すなわち前後軍に授け、太史慈を先手とし、魏延、徐晃などを後陣として、昼夜、金鼓鉄槍を震わせて、陣々の要害に迫って行った。
敵影を見ないうちに、金鼓鉄槍の音は、すでに四方に鳴りひびいた。周瑜の中軍も、徐晃の後ろ備えも、いちどに奮い起って、
「あれこそ、孔明の与えた妙案である。今こそ奪るは絶好の戦機」
と、大声あげて、指揮に当り、獅子奮迅の機を逸しないように、各所に阻めた。
鶏肋
 魏軍は完敗した。これで金鼓鉄槍もすべて鳴りをしずめた。周瑜は、痩せ衰えたお身の上に、また鉄甲を着かさねて、威風あたりを払ったが、
「何たることだ」
と、底知れない敵意に打たれて、喚きもだめよと、左右へ罵った。
すると、程普は、
「丞相のおことばには、其許のため敗れたのでありません。然るに、周瑜の罪は、われらのものより軽い。これを以て、都督に害意なしなどとは、われを盲にした不敬である」
と、耳もかさない。
怒れる周瑜は、程普を床に蹴って、
「さてさて、周瑜の浅慮は、周都督だけの不徳ではない。誰よりも、丞相自身がゆるさんとしておる。この上は、諸将とよく相談のうえ、丁寧に一計をめぐらして来い」と、痛烈に突っぱねた。
程普は、怒りに燃えながら、陸遜の下へ行き、
「都督、それがしの任は、もうしばらく

In [42]:
# 保存
import csv

with open('generated_text.txt', 'w') as f:
  f.write("\n".join(lines))